## Windowed reading and writing

Sometimes, the input data you need to process are too big, to fit into computer's memory. You need to process the data *per-partes* - split into smaller chunks.

This allowes you also to implement parallel processing of the data.

Windows (`rasterio.Window`) are regular matrices of input raster file. They can be also defined with help of two coordinate pairs:

```
((first_row, last_row), (first_column, last_column))
```

or using `rasterio.Window` object

```
Window(first_column, first_row, width, height)
```


Example of data reading

In [ ]:
import rasterio
from rasterio.windows import Window

with rasterio.open('data/B02-2018-05-06.tiff') as src:
    w = src.read(1, window=Window(0, 0, 512, 256))

print(w.shape)


### Blocks

Raster files are able to store data into block-chunks, for more efficient reading and writing in the future. They can be reached using the `block_windows()` function.

In [ ]:

with rasterio.open('data/B02-2018-05-06.tiff') as src:
    for ji, window in src.block_windows(1):
        r = src.read(1, window=window)
        print(r.shape)

You can check the command-line program `gdalinfo` for raster data source details

```
$ gdalinfo data/lsat7_2002_nir.tiff

Driver: GTiff/GeoTIFF
Files: ../../data/lsat7_2002_nir.tiff
Size is 1287, 831
...
Band 1 Block=1287x1 Type=Float32, ColorInterp=Gray
Band 2 Block=1287x1 Type=Float32, ColorInterp=Undefined
Band 3 Block=1287x1 Type=Float32, ColorInterp=Undefined

Vidíme, že náš rastr používá bloky o velikosti 1287x1 pixel - tedy celý řádek. Někdy může být efektivnější celý rastr převzorkovat a změnit nastavení bloků, než se pustíte samotného výpočtu. Toho docílíme nástrojem knihovny GDAL gdalwarp (viz školení Úvod Open Source GIS).

gdalwarp -r mode -co TILED=YES -co BLOCKXSIZE=256 -co BLOCKYSIZE=256 data/lsat7_2002_nir.tiff outputs/lsat7-256-block.tiff

gdalinfo outputs/lsat7-256-block.tiff
...
Band 1 Block=256x256 Type=Float32, ColorInterp=Gray
Band 2 Block=256x256 Type=Float32, ColorInterp=Undefined
Band 3 Block=256x256 Type=Float32, ColorInterp=Undefined
```


### Calculating NDVI using block windows

This example shows, how to calculate NDVI raster data using window-based reading and writing. On large data files, this approach will perform faster.

In [ ]:
import rasterio
from rasterio.windows import Window

with rasterio.open('data/B04-2018-05-06.tiff') as red_file:
    with rasterio.open('data/B08-2018-05-06.tiff') as nir_file:
        
        step = 256
        kwargs = red_file.meta
        kwargs.update(dtype=rasterio.float64, count=1, compress='lzw')

        with rasterio.open('../outputs/ndvi-windows.tif', 'w', **kwargs) as dst:
            slices = [(col_start, row_start, step, step) \
                    for col_start in list(range(0, src.width, 256)) \
                    for row_start in list(range(0, src.height, 256))
        ]

            # we are not going to use block_windows, since the blocks are 8x3117 pixels.
            for slc in slices:
                win = Window(*slc)
                
                nir = red_file.read(1, window=win) # band index starts with 1, not 0
                vis = nir_file.read(1, window=win) 

                ndvi = (1.0*nir - 1.0*vis) / (1.0*nir + 1.0*vis)

                write_win = Window(slc[0], slc[1], ndvi.shape[1], ndvi.shape[0])

                dst.write_band(1, ndvi.astype(rasterio.float64), window=write_win)
                


You can now download data and display in QGIS [ndvi-windows.tif](../outputs/ndvi-windows.tif)

---
[<- NDVI interpretation](03_ndvi-interpretation.ipynb) | [Introduction to OWS ->](../03_ows/00_intro.ipynb)